# Bibliotecas

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import funcoes_modelos as fmod
import funcoes_bow as fbow
import funcoes_cos as fcos

import time

## Configurações

In [2]:
# flag_cpu = True
# if flag_cpu == True:
#     # force CPU (make CPU visible)
#     cpus = tf.config.experimental.list_physical_devices('CPU')
#     print(cpus)
#     tf.config.set_visible_devices([], 'GPU')  # hide the GPU
#     tf.config.set_visible_devices(cpus[0], 'CPU') # unhide potentially hidden CPU
#     tf.config.get_visible_devices()

## Constantes

In [3]:
SEED = 42

# Leitura dos arquivos

In [4]:
arquivos = ['hn_balanceado', 'hn_desbalanceado', 'sn_balanceado', 'sn_desbalanceado']

lista_df = []
for arquivo in arquivos:

    df = pd.read_csv(f"Dados/Datasets/{arquivo}.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df.append(df)

In [5]:
#lista_df[0].dtypes

In [6]:
#print(f"Tamanho dos Datasets:\n\n\t\t| Hard\t| Soft\t|\nBalanceado\t| {lista_df[0].shape[0]}\t| {lista_df[2].shape[0]}\t|\nDesbalanceado\t| {lista_df[1].shape[0]}\t| {lista_df[3].shape[0]}\t|")

# Separando o Dataset

In [7]:
def train_test_valid(df):
    
    X = df[["titulo_1", "titulo_2"]]
    y = df["match"].to_list()
    
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED, stratify = y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.285, random_state = SEED, stratify = y_train_valid)
    
    return X_train, y_train, X_test, y_test, X_valid, y_valid


In [8]:
lista_train_test_valid = []
for df in lista_df:
    lista_train_test_valid.append(train_test_valid(df))

In [9]:
#for nome, dataset in zip(arquivos, lista_train_test_valid):    
#    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

# Experimentos Com Os Modelos

## Funções

In [10]:
def salvar_df_resultado(report, modelo, nome_dataset, tempo_exec):
    
    df_resultado = pd.DataFrame(report).transpose()
    df_resultado['modelo'] = modelo
    df_resultado['dataset'] = nome_dataset
    df_resultado['tempo'] = tempo_exec

    df_resultado.to_csv(f'Dados/Resultados/{modelo}/{nome_dataset}_resultado.csv', index = True)
    
    return df_resultado

## BERT, roBERTa, XLMR e ELECTRA

In [11]:
modelos = ("BERT", "roBERTa",  "XLMR", "ELECTRA")

In [12]:
'''
for modelo in modelos:
    
    lista_df_resultado = []
    
    for nome, dataset in zip(arquivos, lista_train_test_valid):    
        
        X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

        start_time = time.time()
        (nome, historico, y_test, y_pred) = fmod.pipeline_bert(modelo, nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])
        end_time = time.time()
        runtime = end_time - start_time
        
        pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{modelo}/{nome}_historico.csv', index = False)

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, modelo, nome, runtime)

        lista_df_resultado.append(df_resultado)'''

"\nfor modelo in modelos:\n    \n    lista_df_resultado = []\n    \n    for nome, dataset in zip(arquivos, lista_train_test_valid):    \n        \n        X_train, y_train, X_test, y_test, X_valid, y_valid = dataset\n\n        start_time = time.time()\n        (nome, historico, y_test, y_pred) = fmod.pipeline_bert(modelo, nome, X_train[:5], y_train[:5], X_valid[:5], y_valid[:5], X_test[:5], y_test[:5])\n        end_time = time.time()\n        runtime = end_time - start_time\n        \n        pd.DataFrame.from_dict(historico.history).to_csv(f'Dados/Resultados/{modelo}/{nome}_historico.csv', index = False)\n\n        report = classification_report(y_test, y_pred, output_dict = True)\n        \n        df_resultado = salvar_df_resultado(report, modelo, nome, runtime)\n\n        lista_df_resultado.append(df_resultado)"

## BoW Co ocorrencia

In [13]:
modelo = "BoW"
lista_df_resultado = []
fim = 50


for nome, dataset in zip(arquivos, lista_train_test_valid):
    
    print(f"\n\nNome: {nome}\n\n")
    
    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

    start_time = time.time()
    
    name_dataset, y_test, y_pred = fbow.pipeline_rf(nome, X_train, y_train, X_valid, y_valid, X_test, y_test)
    # name_dataset, y_test, y_pred = fbow.pipeline_rf(nome, X_train[:fim], y_train[:fim], X_valid[:fim], y_valid[:fim], X_test[:fim], y_test[:fim])
    
    #print(f"\n\n{nome}\n\n")

    end_time = time.time()
    runtime = end_time - start_time

    report = classification_report(y_test, y_pred, output_dict = True)
    
    df_resultado = salvar_df_resultado(report, modelo, nome, runtime)

    lista_df_resultado.append(df_resultado)



Nome: hn_balanceado




[I 2022-07-13 15:45:57,043] A new study created in memory with name: no-name-c2c44279-7754-4d2d-8c7f-bd2bb25b4dae
[I 2022-07-13 15:46:05,436] Trial 0 finished with value: 0.9092092417832736 and parameters: {'n_estimators': 226, 'max_depth': 8, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9092092417832736.
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf



Nome: hn_desbalanceado




[I 2022-07-13 15:46:26,368] A new study created in memory with name: no-name-d47f7cf2-faf6-41a5-8957-6dffb22aaa71
[I 2022-07-13 15:47:27,233] Trial 0 finished with value: 0.9090909090909091 and parameters: {'n_estimators': 854, 'max_depth': 11, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9090909090909091.
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\t



Nome: sn_balanceado




[I 2022-07-13 15:48:36,020] A new study created in memory with name: no-name-202ac532-93eb-4e43-995b-8ead7a207430
[I 2022-07-13 15:48:44,547] Trial 0 finished with value: 0.9092092417832736 and parameters: {'n_estimators': 418, 'max_depth': 4, 'criterion': 'gini'}. Best is trial 0 with value: 0.9092092417832736.
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf_gp



Nome: sn_desbalanceado




[I 2022-07-13 15:49:07,359] A new study created in memory with name: no-name-1a23a2e1-13a4-44a0-ab11-77ebbb1ae383
[I 2022-07-13 15:49:42,889] Trial 0 finished with value: 0.9090909090909091 and parameters: {'n_estimators': 509, 'max_depth': 10, 'criterion': 'entropy'}. Best is trial 0 with value: 0.9090909090909091.
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mcso\anaconda3\envs\t

## Similaridade de Cosseno

In [15]:
modelo = "Cos"
lista_df_resultado = []
thresh_list = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for nome, dataset in zip(arquivos, lista_train_test_valid):

    X_train, y_train, X_test, y_test, X_valid, y_valid = dataset

    for threshold in thresh_list:
        start_time = time.time()

        name_dataset, y_test, y_pred = fcos.cos_threshold(nome, X_test, y_test, threshold)

        end_time = time.time()
        runtime = end_time - start_time

        report = classification_report(y_test, y_pred, output_dict = True)
        
        df_resultado = salvar_df_resultado(report, f'{modelo}_0{int(threshold*10)}', nome, runtime)

        lista_df_resultado.append(df_resultado)

    

In [16]:
len(lista_df_resultado)

28